# Scenario Analysis
DSCP-Based QoS - Hierarchical 13 Switches (0.5 Mbps, 216% Utilization)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Better style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.figsize': (12, 7),
    'font.size': 13,
    'axes.titlesize': 16,
    'axes.titleweight': 'bold',
    'axes.labelsize': 13,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
    'figure.facecolor': 'white',
    'axes.facecolor': 'white',
    'axes.edgecolor': '#333333',
    'grid.alpha': 0.4,
})

COLORS = {
    'anomaly': '#E53935',  # Red
    'normal': '#1E88E5',   # Blue
    'improvement': '#43A047'  # Green
}

## 1. Load Data

In [ ]:
# Load 3 CSV files
df1 = pd.read_csv('mqtt_metrics_log1.csv')
df2 = pd.read_csv('mqtt_metrics_log2.csv')
df3 = pd.read_csv('mqtt_metrics_log3.csv')

df1['run'] = 1
df2['run'] = 2
df3['run'] = 3

df = pd.concat([df1, df2, df3], ignore_index=True)
df['traffic_type'] = df['device'].apply(lambda x: 'anomaly' if 'anomaly' in str(x) else 'normal')

print(f'Total messages: {len(df):,}')
print(f'Run 1: {len(df1):,} | Run 2: {len(df2):,} | Run 3: {len(df3):,}')

## 2. Calculate Statistics

In [ ]:
# Calculate per-run statistics
def calc_stats(data):
    # Calculate jitter: collect all per-device jitter values, then flat mean
    all_jitter_values = []
    if len(data) > 1:
        for device in data['device'].unique():
            device_data = data[data['device'] == device].sort_values('seq')
            if len(device_data) > 1:
                delays = device_data['delay_ms'].values
                # Filter out NaN values
                delays = delays[~np.isnan(delays)]
                if len(delays) > 1:
                    jitters = np.abs(np.diff(delays))
                    jitters = jitters[np.isfinite(jitters)]  # Filter NaN/Inf
                    all_jitter_values.extend(jitters)
    
    # Handle empty jitter list - return 0.0 instead of NaN
    avg_jitter = float(np.mean(all_jitter_values)) if len(all_jitter_values) > 0 else 0.0
    
    # Handle NaN in delay values
    delay_vals = data['delay_ms'].dropna()
    
    return {
        'messages': len(data),
        'avg_delay': float(delay_vals.mean()) if len(delay_vals) > 0 else 0.0,
        'min_delay': float(delay_vals.min()) if len(delay_vals) > 0 else 0.0,
        'max_delay': float(delay_vals.max()) if len(delay_vals) > 0 else 0.0,
        'std_delay': float(delay_vals.std()) if len(delay_vals) > 1 else 0.0,
        'jitter': avg_jitter
    }

stats = {}
for run in [1, 2, 3]:
    stats[run] = {}
    for ttype in ['anomaly', 'normal']:
        subset = df[(df['run'] == run) & (df['traffic_type'] == ttype)]
        stats[run][ttype] = calc_stats(subset)

# Calculate averages
anomaly_avg_delay = np.mean([stats[r]['anomaly']['avg_delay'] for r in [1,2,3]])
normal_avg_delay = np.mean([stats[r]['normal']['avg_delay'] for r in [1,2,3]])
improvement = (normal_avg_delay - anomaly_avg_delay) / normal_avg_delay * 100

print(f'Anomaly Avg Delay: {anomaly_avg_delay:,.2f} ms ({anomaly_avg_delay/1000:.2f} s)')
print(f'Normal Avg Delay:  {normal_avg_delay:,.2f} ms ({normal_avg_delay/1000:.2f} s)')
print(f'Delay Improvement: {improvement:.2f}%')

## 3. Export Table (Paper Format)

In [ ]:
# Create table in the requested format
def get_metric_values(metric_key):
    """Get values for a metric across all runs and calculate avg/std"""
    anomaly_vals = [stats[r]['anomaly'][metric_key] for r in [1,2,3]]
    normal_vals = [stats[r]['normal'][metric_key] for r in [1,2,3]]
    return {
        '1st_A': anomaly_vals[0], '1st_N': normal_vals[0],
        '2nd_A': anomaly_vals[1], '2nd_N': normal_vals[1],
        '3rd_A': anomaly_vals[2], '3rd_N': normal_vals[2],
        'Avg_A': np.mean(anomaly_vals), 'Avg_N': np.mean(normal_vals),
    }

metrics = [
    ('Messages Received', 'messages'),
    ('Avg Delay (ms)', 'avg_delay'),
    ('Min Delay (ms)', 'min_delay'),
    ('Max Delay (ms)', 'max_delay'),
    ('Std Dev Delay (ms)', 'std_delay'),
    ('Avg Jitter (ms)', 'jitter'),
]

table_data = {'Metric': [m[0] for m in metrics]}
for col in ['1st_Anomaly', '1st_Normal', '2nd_Anomaly', '2nd_Normal', 
            '3rd_Anomaly', '3rd_Normal', 'Avg_Anomaly', 'Avg_Normal']:
    table_data[col] = []

for metric_name, metric_key in metrics:
    vals = get_metric_values(metric_key)
    table_data['1st_Anomaly'].append(vals['1st_A'])
    table_data['1st_Normal'].append(vals['1st_N'])
    table_data['2nd_Anomaly'].append(vals['2nd_A'])
    table_data['2nd_Normal'].append(vals['2nd_N'])
    table_data['3rd_Anomaly'].append(vals['3rd_A'])
    table_data['3rd_Normal'].append(vals['3rd_N'])
    table_data['Avg_Anomaly'].append(vals['Avg_A'])
    table_data['Avg_Normal'].append(vals['Avg_N'])

df_table = pd.DataFrame(table_data)

# Format numbers based on value size (IEEE style)
def format_value(x, metric_name):
    if pd.isna(x):
        return 0.0
    if 'Messages' in metric_name:
        return int(x)  # Integer for message count
    elif x >= 1000:
        return int(round(x))  # No decimal for large values
    else:
        return round(x, 2)  # 2 decimals for small values

# Apply formatting per metric
for col in df_table.columns[1:]:
    df_table[col] = [format_value(val, metric) for val, metric in zip(df_table[col], df_table['Metric'])]

# Save to CSV
df_table.to_csv('table_results.csv', index=False)

# Save to Excel with formatting
try:
    with pd.ExcelWriter('table_results.xlsx', engine='openpyxl') as writer:
        df_table.to_excel(writer, index=False, sheet_name='Results')
        worksheet = writer.sheets['Results']
        for col in worksheet.columns:
            worksheet.column_dimensions[col[0].column_letter].width = 15
    print('Saved: table_results.csv, table_results.xlsx\n')
except:
    print('Saved: table_results.csv (Excel export requires openpyxl)\n')

df_table

In [ ]:
# Create multi-level header table for better visualization
columns = pd.MultiIndex.from_tuples([
    ('', 'Metric'),
    ('1st', 'Anomaly'), ('1st', 'Normal'),
    ('2nd', 'Anomaly'), ('2nd', 'Normal'),
    ('3rd', 'Anomaly'), ('3rd', 'Normal'),
    ('Avg', 'Anomaly'), ('Avg', 'Normal'),
])

df_display = pd.DataFrame(table_data)
df_display.columns = columns
df_display

## 4. Visualizations

In [ ]:
# Fig 1: Delay Comparison per Run
fig, ax = plt.subplots(figsize=(12, 7))

anomaly_delays = [stats[r]['anomaly']['avg_delay'] for r in [1,2,3]]
normal_delays = [stats[r]['normal']['avg_delay'] for r in [1,2,3]]

x = np.arange(3)
width = 0.35

bars1 = ax.bar(x - width/2, anomaly_delays, width, label='Anomaly (DSCP 46)', 
               color=COLORS['anomaly'], edgecolor='black', linewidth=1.2)
bars2 = ax.bar(x + width/2, normal_delays, width, label='Normal (DSCP 0)', 
               color=COLORS['normal'], edgecolor='black', linewidth=1.2)

ax.set_xlabel('Experiment Run', fontweight='bold')
ax.set_ylabel('Average Delay (ms)', fontweight='bold')
ax.set_title('Average End-to-End Delay per Run')
ax.set_xticks(x)
ax.set_xticklabels(['Run 1', 'Run 2', 'Run 3'])
ax.legend(loc='upper right', framealpha=0.95)
ax.set_ylim(0, max(normal_delays) * 1.2)

# Value labels
for bar in bars1:
    h = bar.get_height()
    ax.annotate(f'{h/1000:.1f}s', xy=(bar.get_x() + bar.get_width()/2, h), 
                xytext=(0, 5), textcoords='offset points', ha='center', 
                fontsize=12, fontweight='bold', color=COLORS['anomaly'])
for bar in bars2:
    h = bar.get_height()
    ax.annotate(f'{h/1000:.1f}s', xy=(bar.get_x() + bar.get_width()/2, h), 
                xytext=(0, 5), textcoords='offset points', ha='center', 
                fontsize=12, fontweight='bold', color=COLORS['normal'])

plt.tight_layout()
plt.savefig('fig1_delay_per_run.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# Fig 2: Overall Delay Comparison with Improvement
fig, ax = plt.subplots(figsize=(10, 7))

means = [anomaly_avg_delay, normal_avg_delay]
stds = [np.std(anomaly_delays), np.std(normal_delays)]
colors = [COLORS['anomaly'], COLORS['normal']]

bars = ax.bar(['Anomaly\n(DSCP 46)', 'Normal\n(DSCP 0)'], means, 
              yerr=stds, capsize=8, color=colors, edgecolor='black', linewidth=1.5,
              error_kw={'linewidth': 2, 'capthick': 2})

ax.set_ylabel('Average Delay (ms)', fontweight='bold')
ax.set_title('Overall Average Delay Comparison')

# Value labels
for bar, mean, color in zip(bars, means, colors):
    ax.annotate(f'{mean/1000:.1f}s\n({mean:,.0f} ms)', 
                xy=(bar.get_x() + bar.get_width()/2, mean),
                xytext=(0, 15), textcoords='offset points', 
                ha='center', fontsize=13, fontweight='bold')

# Improvement box
ax.annotate(f'⬇ {improvement:.1f}% Improvement', 
            xy=(0.5, 0.92), xycoords='axes fraction',
            ha='center', fontsize=14, fontweight='bold', color='white',
            bbox=dict(boxstyle='round,pad=0.5', facecolor=COLORS['improvement'], 
                      edgecolor='darkgreen', linewidth=2))

# Draw improvement arrow
ax.annotate('', xy=(0, anomaly_avg_delay), xytext=(1, normal_avg_delay),
            arrowprops=dict(arrowstyle='<->', color=COLORS['improvement'], lw=2))

plt.tight_layout()
plt.savefig('fig2_delay_comparison.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# Fig 3: Box Plot - Delay Distribution
fig, ax = plt.subplots(figsize=(10, 7))

anomaly_data = df[df['traffic_type']=='anomaly']['delay_ms']
normal_data = df[df['traffic_type']=='normal']['delay_ms']

bp = ax.boxplot([anomaly_data, normal_data], 
                labels=['Anomaly (DSCP 46)', 'Normal (DSCP 0)'],
                patch_artist=True, widths=0.6,
                boxprops=dict(linewidth=2),
                whiskerprops=dict(linewidth=2),
                capprops=dict(linewidth=2),
                medianprops=dict(linewidth=2.5, color='black'),
                flierprops=dict(marker='o', markersize=4, alpha=0.5))

bp['boxes'][0].set_facecolor(COLORS['anomaly'])
bp['boxes'][0].set_alpha(0.7)
bp['boxes'][1].set_facecolor(COLORS['normal'])
bp['boxes'][1].set_alpha(0.7)

ax.set_ylabel('Delay (ms)', fontweight='bold')
ax.set_title('Delay Distribution')

# Add median annotations
medians = [anomaly_data.median(), normal_data.median()]
for i, median in enumerate(medians):
    ax.annotate(f'Median: {median/1000:.1f}s', xy=(i+1, median), 
                xytext=(30, 0), textcoords='offset points',
                fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('fig3_delay_boxplot.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# Fig 4: CDF
fig, ax = plt.subplots(figsize=(12, 7))

for ttype, color, label in [('anomaly', COLORS['anomaly'], 'Anomaly (DSCP 46)'),
                             ('normal', COLORS['normal'], 'Normal (DSCP 0)')]:
    data = df[df['traffic_type']==ttype]['delay_ms'].sort_values()
    cdf = np.arange(1, len(data)+1) / len(data)
    step = max(1, len(data)//5000)
    ax.plot(data.values[::step], cdf[::step], label=label, color=color, linewidth=2.5)
    
    # Mark 50th and 90th percentile
    p50 = np.percentile(data, 50)
    p90 = np.percentile(data, 90)
    ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
    ax.axhline(y=0.9, color='gray', linestyle='--', alpha=0.5)

ax.set_xlabel('Delay (ms)', fontweight='bold')
ax.set_ylabel('Cumulative Probability (CDF)', fontweight='bold')
ax.set_title('Cumulative Distribution Function of End-to-End Delay')
ax.legend(loc='lower right', fontsize=12)
ax.grid(True, alpha=0.4)

# Add percentile labels
ax.text(ax.get_xlim()[1]*0.98, 0.5, '50th percentile', ha='right', va='bottom', fontsize=10, color='gray')
ax.text(ax.get_xlim()[1]*0.98, 0.9, '90th percentile', ha='right', va='bottom', fontsize=10, color='gray')

plt.tight_layout()
plt.savefig('fig4_delay_cdf.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# Fig 5: Messages Received (Throughput)
fig, ax = plt.subplots(figsize=(12, 7))

msg_anomaly = [stats[r]['anomaly']['messages'] for r in [1,2,3]]
msg_normal = [stats[r]['normal']['messages'] for r in [1,2,3]]

x = np.arange(3)
width = 0.35

bars1 = ax.bar(x - width/2, msg_anomaly, width, label='Anomaly (DSCP 46)', 
               color=COLORS['anomaly'], edgecolor='black', linewidth=1.2)
bars2 = ax.bar(x + width/2, msg_normal, width, label='Normal (DSCP 0)', 
               color=COLORS['normal'], edgecolor='black', linewidth=1.2)

ax.set_xlabel('Experiment Run', fontweight='bold')
ax.set_ylabel('Messages Received', fontweight='bold')
ax.set_title('Messages Received per Run')
ax.set_xticks(x)
ax.set_xticklabels(['Run 1', 'Run 2', 'Run 3'])
ax.legend(loc='upper right')

for bar in bars1 + bars2:
    h = bar.get_height()
    ax.annotate(f'{h:,}', xy=(bar.get_x() + bar.get_width()/2, h), 
                xytext=(0, 5), textcoords='offset points', ha='center', fontsize=11)

# Add ratio annotation
ratio = np.mean(msg_anomaly) / np.mean(msg_normal)
ax.annotate(f'Anomaly receives {ratio:.1f}x more messages', 
            xy=(0.5, 0.02), xycoords='axes fraction',
            ha='center', fontsize=12, style='italic')

plt.tight_layout()
plt.savefig('fig5_messages_received.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

## 5. Summary

In [ ]:
print('='*70)
print('SCENARIO 6 SUMMARY: HIGH CONGESTION')
print('='*70)
print(f'''
Configuration:
  - Topology     : Hierarchical 3-Tier (13 switches)
  - Bandwidth    : 0.5 Mbps
  - Utilization  : 216%
  - Publishers   : 18 (9 anomaly + 9 normal)
  - Runs         : 3

Results:
  - Anomaly Avg Delay : {anomaly_avg_delay:,.0f} ms ({anomaly_avg_delay/1000:.1f} s)
  - Normal Avg Delay  : {normal_avg_delay:,.0f} ms ({normal_avg_delay/1000:.1f} s)
  - Delay Improvement : {improvement:.1f}%
  
  - Anomaly Messages  : {np.mean(msg_anomaly):,.0f} (avg per run)
  - Normal Messages   : {np.mean(msg_normal):,.0f} (avg per run)
  - Throughput Ratio  : {ratio:.1f}x

Files Generated:
  - table_results.csv
  - fig1_delay_per_run.png
  - fig2_delay_comparison.png
  - fig3_delay_boxplot.png
  - fig4_delay_cdf.png
  - fig5_messages_received.png
''')
print('='*70)